In [1]:
from huggingface_hub import notebook_login

notebook_login()


In [2]:
from datasets import load_dataset

imdb = load_dataset("imdb")

print(imdb)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [11]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [6]:
import evaluate

accuracy = evaluate.load("accuracy")

In [7]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [8]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [9]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
training_args = TrainingArguments(
    output_dir="models/distilbert-base-uncased-imdb_finetune",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    device='mps'
)

trainer.train()

  0%|          | 0/3126 [00:00<?, ?it/s]

{'loss': 0.3268, 'grad_norm': 10.57805061340332, 'learning_rate': 1.6801023672424827e-05, 'epoch': 0.32}
{'loss': 0.2458, 'grad_norm': 8.134000778198242, 'learning_rate': 1.3602047344849649e-05, 'epoch': 0.64}
{'loss': 0.2206, 'grad_norm': 12.854681015014648, 'learning_rate': 1.0403071017274472e-05, 'epoch': 0.96}


  0%|          | 0/1563 [00:00<?, ?it/s]

{'eval_loss': 0.20592010021209717, 'eval_accuracy': 0.9208, 'eval_runtime': 502.7527, 'eval_samples_per_second': 49.726, 'eval_steps_per_second': 3.109, 'epoch': 1.0}
{'loss': 0.1584, 'grad_norm': 11.682337760925293, 'learning_rate': 7.204094689699297e-06, 'epoch': 1.28}
{'loss': 0.1394, 'grad_norm': 21.69116973876953, 'learning_rate': 4.005118362124121e-06, 'epoch': 1.6}
{'loss': 0.1402, 'grad_norm': 4.037898063659668, 'learning_rate': 8.061420345489445e-07, 'epoch': 1.92}


  0%|          | 0/1563 [00:00<?, ?it/s]

{'eval_loss': 0.23387768864631653, 'eval_accuracy': 0.93196, 'eval_runtime': 468.3365, 'eval_samples_per_second': 53.38, 'eval_steps_per_second': 3.337, 'epoch': 2.0}
{'train_runtime': 4237.888, 'train_samples_per_second': 11.798, 'train_steps_per_second': 0.738, 'train_loss': 0.20270711858533394, 'epoch': 2.0}


No files have been modified since last commit. Skipping to prevent empty commit.


TrainOutput(global_step=3126, training_loss=0.20270711858533394, metrics={'train_runtime': 4237.888, 'train_samples_per_second': 11.798, 'train_steps_per_second': 0.738, 'total_flos': 6556904415524352.0, 'train_loss': 0.20270711858533394, 'epoch': 2.0})

In [14]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/liukai1987/distilbert-base-uncased-imdb_finetune/commit/1d259e685492107e53915c9ccf902c2fb6a521df', commit_message='End of training', commit_description='', oid='1d259e685492107e53915c9ccf902c2fb6a521df', pr_url=None, pr_revision=None, pr_num=None)

In [15]:
repo_name = "https://github.com/liukai1919/LLMProject"
training_args = TrainingArguments(
   output_dir=repo_name,
   push_to_hub=True,
)

In [19]:
from accelerate import Accelerator

# Reinitialize the Accelerator
accelerator = Accelerator()

In [20]:
trainer.evaluate()

  0%|          | 0/1563 [00:00<?, ?it/s]

{'eval_loss': 0.20592010021209717,
 'eval_accuracy': 0.9208,
 'eval_runtime': 473.1839,
 'eval_samples_per_second': 52.834,
 'eval_steps_per_second': 3.303,
 'epoch': 2.0}

In [21]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/liukai1987/distilbert-base-uncased-imdb_finetune/commit/8509928524904217b4a2f529c5d5db16bd1cfa06', commit_message='End of training', commit_description='', oid='8509928524904217b4a2f529c5d5db16bd1cfa06', pr_url=None, pr_revision=None, pr_num=None)

In [24]:
from transformers import pipeline

# Define the data
data = [
    'Total surprise, enjoyed this "Nollywood" flick. I started the movie curious to see more of Nigeria. Sure it has its production flaws, but as we watched, didn\'t want to turn off. Frank is a very likeable character and you feel his highs and lows. Now will look for more with this actor.'
]



pipe = pipeline("text-classification", model="liukai1987/distilbert-base-uncased-imdb_finetune", device='mps')

# Use the pipeline to make predictions
predictions = pipe(data)

# Print the predictions
print(predictions)


[{'label': 'POSITIVE', 'score': 0.933591902256012}]
